In [3]:
import numpy as np
import pandas as pd
import warnings

# EDA 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats
from scipy.stats import skew
from scipy.stats import spearmanr

# Learning algorithms
import sklearn
from sklearn.linear_model import *
from sklearn.svm import SVR
from sklearn.cluster import KMeans


In [4]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
ss = pd.read_csv('data/sample_submission.csv')

In [ ]:
# 컬럼명 영문으로 변경하기
train_df.columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
test_df.columns = ['num','datetime','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']

In [ ]:
train_df.shape

In [ ]:
train_df.head()

비가 오는 날과 맑은 날의 전력 사용량에 차이가 날 것 이라 생각이 들었고, 날씨와 전력 사용량의 상관 관계를 알아보면 좋겠다는 생각이 들었습니다.

그래서 weather이라는 컬럼을 새로 만들고, 비오는 날을 0, 흐린 날을 0.5, 맑은 날을 1로 값을 주었습니다.

- 비오는날
    - 강수량 > 0
- 흐린 날
    - 강수량 = 0, 일조량
- 맑은 날
    - 강수량 = 0, 일조량 = 1
    - 강수량 = 0, 일조량 = 0 , 습도 
    
흐린 날과, 맑은 날의 데이터를 뽑기 위해 일조량과 습도를 알아내고 싶었습니다.

그래서 비오는 날의 dataframe과 비가 오지 않는 날의 dataframe을 뽑아내고, 습도, 강수량, 일조에 대한 데이터를 뽑아 보았습니다.

In [ ]:
rainy_df = train_df[train_df['precipitation']>0]

print(rainy_df.shape)

In [ ]:
not_rainy_df = train_df[train_df['precipitation']==0]

print(not_rainy_df.shape)

122400개의 데이터 중, 비가 오는 data는 18927개, 비가 오지 않는 data는 103473개가 있습니다.

비오는날의 일조량, 습도를 뽑아 보는데, 해가 뜬 시간과 해가 저문 시간에 따라 일조량과 습도가 다를 것 같아, 06\~18시 까지는 day time, 00\~06, 18\~23는 night time으로 나누어 일조량과 습도를 뽑아보았습니다.

# daytime에서 RainyDay 일조량, 습도 데이터

In [ ]:
daytime_rainy_df = rainy